## Feature Engineering and CV based on 5-th Place Solutions

`param = {'objective': 'multi:softprob', 
         'eta': 0.05, 
         'max_depth': 8,
         'silent': 1, 
         'num_class': len(target_cols),
         'eval_metric': 'mlogloss',
         'min_child_weight': 1,
         'subsample': 0.7,
         'colsample_bytree': 0.7,
         'seed': 0}`
         
`n_repeats=2
n_trees = 150`

Train on the last three months

New in this notebook:
- Create training data for all months (2015-02-28 to 2016-04-28) and validate on 2016-05-28

In [1]:
from santander_helper import *

Load all months' data

In [3]:
x_train_list = []
y_train_list = []
w_train_list = []
fixed_lag = 6
for i, m in tqdm.tqdm_notebook(enumerate(month_list), total=len(month_list)):
    if m in ['2015-01-28', '2016-06-28']:
        continue
    x_tmp, y_tmp, w_tmp = create_train(m, max_lag=i, fixed_lag=fixed_lag, pattern_flag=True)
    x_train_list.append(x_tmp)
    y_train_list.append(y_tmp)
    w_train_list.append(w_tmp)
    print(len(x_train_list), m)
del x_tmp, y_tmp, w_tmp
gc.collect()

1 2015-02-28
2 2015-03-28
3 2015-04-28
4 2015-05-28
5 2015-06-28
6 2015-07-28
7 2015-08-28
8 2015-09-28
9 2015-10-28
10 2015-11-28
11 2015-12-28
12 2016-01-28
13 2016-02-28
14 2016-03-28
15 2016-04-28
16 2016-05-28



268

Prepare the last few months for train and validation

In [5]:
# Prepare for train and validation
x_val = x_train_list[-1]
y_val = y_train_list[-1]
w_val = w_train_list[-1]

x_train = x_train_list[10]
y_train = y_train_list[10]
w_train = w_train_list[10]

gc.collect()

0

In [6]:
param = {'objective': 'multi:softprob', 
         'eta': 0.1, 
         'max_depth': 8,
         'silent': 1, 
         'num_class': len(target_cols),
         'eval_metric': 'merror',
         'min_child_weight': 10,
         'lambda': 5,
         'subsample': 0.7,
         'colsample_bytree': 0.7,
         'seed': 0}

n_rows = None # number of rows in train dataset, to simplify testing, always set to None
n_repeats = 1
n_trees = 10
train = {'x': x_train.iloc[:n_rows, :], 'y': y_train.iloc[:n_rows], 'w': w_train.iloc[:n_rows]}
val = {'x': x_val.iloc[:n_rows, :], 'y': y_val.iloc[:n_rows], 'w': w_val.iloc[:n_rows]}
df, clfs, running_time = cv_all_month(param, train, val, n_features=350, num_boost_round=n_trees, 
    n_repeats=n_repeats, random_state=0, verbose_eval=True)

[0]	train-merror:0.312874	val-merror:0.322783	train-MAP@7:0	val-MAP@7:0.858613
[1]	train-merror:0.306694	val-merror:0.316606	train-MAP@7:0	val-MAP@7:0.86648
[2]	train-merror:0.302832	val-merror:0.31373	train-MAP@7:0	val-MAP@7:0.869181
[3]	train-merror:0.300315	val-merror:0.311853	train-MAP@7:0	val-MAP@7:0.870595
[4]	train-merror:0.299282	val-merror:0.310692	train-MAP@7:0	val-MAP@7:0.871565
[5]	train-merror:0.296887	val-merror:0.311568	train-MAP@7:0	val-MAP@7:0.871126
[6]	train-merror:0.294884	val-merror:0.311915	train-MAP@7:0	val-MAP@7:0.871086
[7]	train-merror:0.294178	val-merror:0.312029	train-MAP@7:0	val-MAP@7:0.871132
[8]	train-merror:0.292392	val-merror:0.310079	train-MAP@7:0	val-MAP@7:0.872557
[9]	train-merror:0.291014	val-merror:0.308935	train-MAP@7:0	val-MAP@7:0.873403
Repeat 0, validate score = 0.309, running time = 1.248 min
Score mean = 0.309, std = nan


Train on 1603-1605

In [ ]:
n_rows = None # number of rows in train dataset, to simplify testing, always set to None
n_repeats = 1
n_trees = 5
train = {'x': x_train.iloc[:n_rows, :], 'y': y_train.iloc[:n_rows], 'w': w_train.iloc[:n_rows]}
# val = {'x': x_val.iloc[:n_rows, :], 'y': y_val.iloc[:n_rows], 'w': w_val.iloc[:n_rows]}
clfs, running_time = cv_all_month(param, train, n_features=350, num_boost_round=n_trees, 
    n_repeats=n_repeats, random_state=47, verbose_eval=True)

[0]	train-merror:0.312713	train-MAP@7:0
[1]	train-merror:0.306654	train-MAP@7:0
[2]	train-merror:0.30352	train-MAP@7:0


Predict on 1606

In [5]:
x_test = create_test(month='2016-06-28', max_lag=17, fixed_lag=6, pattern_flag=True)

In [6]:
simulation_name = 'p2'
y_pred, y_sub = predict_all_month(model_dict=clfs, x_test=x_test, 
    sub_name='eda_4_34_{}.csv.gz'.format(simulation_name), n_features=350, n_trees=n_trees)
save_pickle('eda_4_34_results_{}.pickle'.format(simulation_name), (clfs, running_time, y_pred, y_sub))

Load predictions based on 1506 and all months

In [7]:
y_pred2 = load_pickle('parameter_tune_eda_4_32_p4.pickle')
y_pred2 = y_pred2[-4]

y_pred3 = load_pickle('eda_4_33_results_p2.pickle')
y_pred3 = y_pred3[2]

y_pred4 = load_pickle('eda_4_34_results_p1.pickle')
y_pred4 = y_pred4[2]

Combine three models

In [18]:
y_pred_final = np.concatenate((y_pred, y_pred2, y_pred3))

In [20]:
y_pred_final = np.mean(y_pred_final, axis=0)

In [22]:
y_sub = np.argsort(y_pred_final, axis=1)
y_sub = np.fliplr(y_sub)[:, :7]
# Prepare submission
test_id = x_test.loc[:, 'ncodpers'].values
y_sub = [' '.join([target_cols[k] for k in pred]) for pred in y_sub]
y_sub = pd.DataFrame({'ncodpers': test_id, 'added_products': y_sub})
y_sub.to_csv('eda_4_34_p1+eda_4_33_p1+eda_4_32_p4.csv.gz', compression='gzip', index=False)